In [6]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import glob
import json

import pandas as pd

In [8]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = True if value == 'True' else False

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())


        big_data = big_data.append(d, ignore_index=True)

# files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
# i = 0
# for filename in files:
#     #print(f'parsing : {filename}')
#     json_file = filename.replace('backtest_result.txt','live_config.json')
#     with open(json_file) as f:
#         live_config = json.load(f)
#         config_type = live_config['config_name']
#
#     with open(filename, 'r') as file:
#         data = file.read().split('\n')
#         d = {'file': filename.replace('backtests/binance_production','')}
#         current_state = 'long'
#         for l in data:
#             l = l.split('|')
#             if len(l) == 4:
#                 key = l[1].strip()
#                 value = l[2].strip()
#
#                 if key == 'Passivbot Version':
#                     d['v'] = value
#                 if key == 'Symbol':
#                     d['s'] = value
#
#                 if key == 'Long':
#                     current_state = 'long'
#                     d['long'] = bool(value) or False
#                 if key == 'Short':
#                     current_state = 'short'
#                     d['short'] = bool(value) or False
#
#                 if key == 'Total gain' and current_state == 'long':
#                     d['tgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'long':
#                     d['adgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'long':
#                     d['bnkrl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'long':
#                     d['ebrl'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'long':
#                     d['maxsl'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'long':
#                     d['pamaxl'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'long':
#                     d['pastdl'] = float(value.strip())
#
#
#                 if key == 'Total gain' and current_state == 'short':
#                     d['tgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'short':
#                     d['adgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'short':
#                     d['bnkrs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'short':
#                     d['ebrs'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'short':
#                     d['maxss'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'short':
#                     d['pamaxs'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'short':
#                     d['pastds'] = float(value.strip())
#
#         d['prod'] = True
#         big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_8784/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_8784/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_8784/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_8784/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=Tr

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod
0,\1000SHIBUSDT\plots\2022-04-03T194816\backtest...,5.5,1000SHIBUSDT,True,52.53,0.1780,0.066776,0.644511,100.0,0.8613,12.442375,True,-96.020,-1.88000,0.146280,0.802360,5.447,0.10870,155.190833,NaN
1,\1000SHIBUSDT\plots\2022-04-03T204145\backtest...,5.5,1000SHIBUSDT,True,26.73,0.0850,0.070106,0.830417,100.0,0.9153,6.572917,True,-95.820,-1.88000,0.148225,0.803949,5.404,0.09669,155.191667,NaN
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,5.5,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.0,0.7673,12.442375,True,-91.820,-1.64000,0.124514,0.747041,5.959,0.08869,180.079167,True
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
4,\1000XECUSDT\plots\2022-04-06T181135\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,\ZENUSDT\plots\2022-04-08T000448\backtest_resu...,5.5,ZENUSDT,True,78.47,0.2110,0.129356,0.981507,100.0,0.8117,14.342375,True,8.361,0.19300,0.132544,0.571631,19.640,0.25670,47.980000,False
382,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,5.5,ZILUSDT,True,11.86,0.0150,0.213304,1.171540,100.0,0.8399,35.809042,True,-90.810,-1.64000,0.128495,0.776683,5.828,0.07862,33.732625,True
383,\ZILUSDT\plots\2022-04-13T093427\backtest_resu...,5.5,ZILUSDT,True,24.10,0.0730,0.077813,0.534668,100.0,0.7921,13.360417,True,-90.650,-1.64000,0.125380,0.780698,5.950,0.07147,33.434708,False
384,\ZRXUSDT\plots\2022-04-07T125157\backtest_resu...,5.5,ZRXUSDT,True,57.37,0.1690,0.114260,0.708101,100.0,0.8835,22.423625,True,20.130,0.00183,0.141192,0.532109,93.390,0.63730,47.927917,True


Now filter the dataframe

In [11]:
# allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
#                    "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
#                    "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
#                    "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
#                    "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
#                    "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
#                    "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
#                    "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
#                    "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
#                    "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
#                    "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
#                    "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
#                    "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
#                    "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
#                    "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
#                    "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
#                    "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
#                    "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
#                    "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
#                    "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = [
    #"OMG","DUSK","GTC"
    # "LRC",
    # "--------------------------",
    # "SNX", "RSR",
    # "--------------------------",
    # "CELO", "ROSE", "OCEAN","SXP","DENT"
    # "SFP","ENS","DODO","AKRO","REEF","NEO",
    # "----------------------------------------",
    # "1000XEC","AKRO","ALPHA","ANKR","BAKE","BAND","BEL","BTS","C98","COMP",
    "-----------------------------------",
    # "DGB","DOGO","FLM","FLOW","HBAR",
    # "-----------------------------------",
    # "HOT","ICX","IOST","IOTA","KAVA","KLAY","LINA","LIT","MKR","MTL","NEO","NKN","OGN","ONT"
    #"QTUM","REEF","RLC","RVN","SC","SFP","STMX","TOMO","TRB","UNFI","WAVES", "ZIL", "ZRX"
    #"COTI"
    #"RUNE","UNI"
    #"BAT", "BAL", "SRM", "RUNE",
    #"BAT", "BAL", "SRM", "RUNE" , 'STORJ','UNI','FTM','FIL','LRC','CVC',"CTK","SKL","SAND","XEM"
    "XRP"
]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,...,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod,adgt_%
365,\XRPUSDT\plots\2022-04-02T215823\backtest_resu...,5.5,XRPUSDT,True,17.80,0.0669,0.090390,1.012870,100.000,0.8298,...,True,14.1600,0.0628,0.086507,0.401661,100.00,0.8590,20.811125,NaN,0.1297
366,\XRPUSDT\plots\2022-04-02T220407\backtest_resu...,5.5,XRPUSDT,True,10.95,0.0362,0.070355,0.680261,100.000,0.9241,...,True,7.5180,0.0346,0.090093,0.407668,100.00,0.8615,34.021542,NaN,0.0708
367,\XRPUSDT\plots\2022-04-02T220532\backtest_resu...,5.5,XRPUSDT,True,-47.15,-1.8800,0.013518,0.074892,5.986,0.9461,...,True,11.6900,0.0278,0.108543,0.423810,100.00,0.8511,118.224167,NaN,-1.8522
368,\XRPUSDT\plots\2022-04-03T071614\backtest_resu...,5.5,XRPUSDT,True,17.80,0.0669,0.090390,1.012870,100.000,0.8298,...,True,14.1600,0.0628,0.086507,0.401661,100.00,0.8590,20.811125,NaN,0.1297
369,\XRPUSDT\plots\2022-04-03T085436\backtest_resu...,5.5,XRPUSDT,True,10.95,0.0362,0.070355,0.680261,100.000,0.9241,...,True,7.5180,0.0346,0.090093,0.407668,100.00,0.8615,34.021542,NaN,0.0708
370,\XRPUSDT\plots\2022-04-07T122119\backtest_resu...,5.5,XRPUSDT,True,38.19,0.1210,0.065995,0.830946,100.000,0.7749,...,True,0.7041,0.0298,0.126403,0.578940,76.63,0.5702,34.033333,True,0.1508
371,\XRPUSDT\plots\2022-04-23T223904\backtest_resu...,5.5,XRPUSDT,True,44.43,0.1270,0.050968,0.459087,100.000,0.8410,...,True,-0.6001,0.0386,0.160641,0.605579,64.69,0.5172,48.026250,False,0.1656
372,\XRPUSDT\plots\2022-04-23T224046\backtest_resu...,5.5,XRPUSDT,True,24.78,0.0643,0.075890,0.645269,100.000,0.8907,...,True,7.5760,0.0409,0.109422,0.582678,75.84,0.5660,27.393750,False,0.1052
373,\XRPUSDT\plots\2022-04-23T225156\backtest_resu...,5.5,XRPUSDT,True,38.19,0.1120,0.065315,0.818090,100.000,0.7768,...,True,4.6040,0.0422,0.124802,0.578971,76.63,0.5701,34.033333,True,0.1542


In [12]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_8784/2052413415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
365,\XRPUSDT\plots\2022-04-02T215823\backtest_resu...,NaN,XRPUSDT,0.1297,0.0669,0.0628,100.000,100.00,0.8298,0.8590,27.885417,1.012870,0.090390,20.811125,0.401661,0.086507
366,\XRPUSDT\plots\2022-04-02T220407\backtest_resu...,NaN,XRPUSDT,0.0708,0.0362,0.0346,100.000,100.00,0.9241,0.8615,14.947208,0.680261,0.070355,34.021542,0.407668,0.090093
367,\XRPUSDT\plots\2022-04-02T220532\backtest_resu...,NaN,XRPUSDT,-1.8522,-1.8800,0.0278,5.986,100.00,0.9461,0.8511,227.831250,0.074892,0.013518,118.224167,0.423810,0.108543
370,\XRPUSDT\plots\2022-04-07T122119\backtest_resu...,True,XRPUSDT,0.1508,0.1210,0.0298,100.000,76.63,0.7749,0.5702,15.477083,0.830946,0.065995,34.033333,0.578940,0.126403
371,\XRPUSDT\plots\2022-04-23T223904\backtest_resu...,False,XRPUSDT,0.1656,0.1270,0.0386,100.000,64.69,0.8410,0.5172,15.477083,0.459087,0.050968,48.026250,0.605579,0.160641
372,\XRPUSDT\plots\2022-04-23T224046\backtest_resu...,False,XRPUSDT,0.1052,0.0643,0.0409,100.000,75.84,0.8907,0.5660,20.104167,0.645269,0.075890,27.393750,0.582678,0.109422
373,\XRPUSDT\plots\2022-04-23T225156\backtest_resu...,True,XRPUSDT,0.1542,0.1120,0.0422,100.000,76.63,0.7768,0.5701,15.477083,0.818090,0.065315,34.033333,0.578971,0.124802
